In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import WebDriverException
from IPython.display import clear_output

from selenium.webdriver.common.keys import Keys
from IPython.display import clear_output

In [2]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

def createDriver():
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1625x2444")
    # options.headless = True
    return webdriver.Chrome(options = chrome_options)

In [3]:
browser = createDriver()

In [4]:
url_base = "https://www.ilusion.com/moda.html"

In [5]:
browser.get(url_base)

In [31]:
#coneccion(url_base)

In [7]:
LINKS =[
        "https://www.ilusion.com/babydolls.html",
        "https://www.ilusion.com/bras.html",
        "https://www.ilusion.com/lenceria-tops-y-camisetas.html",
        "https://www.ilusion.com/panties.html",
        "https://www.ilusion.com/fondos.html",
        "https://www.ilusion.com/control.html",
        "https://www.ilusion.com/lenceria-ropa-de-dormir.html",
        "https://www.ilusion.com/deportiva-bras.html",
        "https://www.ilusion.com/deportiva-tops-y-camisetas.html",
        "https://www.ilusion.com/deportiva-leggings.html",
        "https://www.ilusion.com/deportiva-chamarras.html",
        "https://www.ilusion.com/ropa-exterior.html"
]

# Escrapeo de los links 

In [19]:
hrefs_list = []
for link in LINKS:
    browser.get(link)
    while True:
        
        for i_ref, h_ref in enumerate(browser.find_elements_by_css_selector('a.product.photo.product-item-photo.content-loader')):
            hrefs_list.append([i_ref+1, h_ref.get_attribute('href'),link])
        try:
            browser.find_elements_by_css_selector('.item.pages-item-next')[1].click()
        except:
            break
    

In [21]:
len(hrefs_list)

428

In [18]:

#maincontent > div.columns > div.column.main > div.toolbar.toolbar-products > div.pages > ul > li.item.pages-item-next > a

'Page\nSiguiente'

In [33]:
pos = 0
cantT = 0
links_productos = []
for y in LINKS:
    cant = 0
    paginacion = 1
    while True:
        try:
            browser.get(LINKS[pos][1])
            break
        except:
            browser.quit()
            browser = createDriver()
            browser.get(LINKS[pos][1])
            browser.delete_all_cookies()
    flag = False
    while flag == False:
        for x in browser.find_elements_by_css_selector("#maincontent > div.columns > div.column.main > div.products.wrapper.grid.products-grid > ol > li > div > div.img-box.configurable > a"):
            link = x.get_attribute("href")
            url_paginacion = browser.current_url
            links_productos.append([cantT,link,url_paginacion])
            cantT = cantT + 1
            cant = cant + 1
        try:
            browser.find_element_by_css_selector("#maincontent > div.columns > div.column.main > div.toolbar.toolbar-products > div.pages > ul > li.item.pages-item-next > a").click()
            paginacion = 1 + paginacion
        except:
            flag = True
    print(f"Links escrapeados {cant} de la pagina {LINKS[pos][1]} con {paginacion} paginaciones")
    pos = pos + 1
print("-------------------------------------")
print(f"Escrapeo finalizado con un total de: {cantT}")

Links escrapeados 26 de la pagina https://www.ilusion.com/babydolls.html con 3 paginaciones
Links escrapeados 115 de la pagina https://www.ilusion.com/bras.html con 10 paginaciones
Links escrapeados 12 de la pagina https://www.ilusion.com/lenceria-tops-y-camisetas.html con 1 paginaciones
Links escrapeados 130 de la pagina https://www.ilusion.com/panties.html con 11 paginaciones
Links escrapeados 24 de la pagina https://www.ilusion.com/fondos.html con 2 paginaciones
Links escrapeados 35 de la pagina https://www.ilusion.com/control.html con 3 paginaciones
Links escrapeados 57 de la pagina https://www.ilusion.com/lenceria-ropa-de-dormir.html con 6 paginaciones
Links escrapeados 5 de la pagina https://www.ilusion.com/deportiva-bras.html con 1 paginaciones
Links escrapeados 8 de la pagina https://www.ilusion.com/deportiva-tops-y-camisetas.html con 1 paginaciones
Links escrapeados 3 de la pagina https://www.ilusion.com/deportiva-leggings.html con 1 paginaciones
Links escrapeados 2 de la pagi

In [34]:
links_productos

[[0,
  'https://www.ilusion.com/babydolls/teddy-7943.html',
  'https://www.ilusion.com/babydolls.html'],
 [1,
  'https://www.ilusion.com/babydolls/babydoll-42188.html',
  'https://www.ilusion.com/babydolls.html'],
 [2,
  'https://www.ilusion.com/babydolls/babydoll-42187.html',
  'https://www.ilusion.com/babydolls.html'],
 [3,
  'https://www.ilusion.com/babydolls/babydoll-42189.html',
  'https://www.ilusion.com/babydolls.html'],
 [4,
  'https://www.ilusion.com/babydolls/babydoll-tipo-halter-42112.html',
  'https://www.ilusion.com/babydolls.html'],
 [5,
  'https://www.ilusion.com/babydolls/babydoll-42186.html',
  'https://www.ilusion.com/babydolls.html'],
 [6,
  'https://www.ilusion.com/babydolls/babydoll-42184.html',
  'https://www.ilusion.com/babydolls.html'],
 [7,
  'https://www.ilusion.com/babydolls/liguero-1176.html',
  'https://www.ilusion.com/babydolls.html'],
 [8,
  'https://www.ilusion.com/babydolls/teddy-42181.html',
  'https://www.ilusion.com/babydolls.html'],
 [9,
  'https://

In [94]:
browser.delete_all_cookies()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=56199): Max retries exceeded with url: /session/bf123fa4b3e729c57313cb797de09175/cookie (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe92f28af98>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [11]:
len(links_productos)

0

In [ ]:
#barra al revex + . \.

In [208]:
pos = 0
item = []
for x in links_productos:
    
    while True:
        try:
            browser.get(links_productos[pos][1])
            break
        except:
            browser.quit()
            browser = createDriver()
            browser.get(links_productos[pos][1])
            browser.delete_all_cookies()
    try:
        for y in browser.find_element_by_css_selector("div.swatch-attribute-options.clearfix").find_elements_by_tag_name("div"):
            y.click()
            time.sleep(2)
            try:
                imagen = browser.find_element_by_class_name("fotorama__img").get_attribute("src")
            except:
                time.sleep(2)
                imagen = browser.find_element_by_class_name("fotorama__img").get_attribute("src")
            id_prodcuto = browser.find_element_by_class_name("base").text.split(" ")[1]
            descripcion = browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split('"')[21]
            #descripcion = browser.find_element_by_class_name("base").text.split(" ")[0]
            color = browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text").split('"')[31]
            #color = browser.find_element_by_css_selector("span.swatch-attribute-selected-option").get_attribute("innerHTML")
            precio = browser.find_element_by_class_name("price").text

            producto_link = links_productos[pos][1]
            producto_id = browser.find_element_by_id("product-addtocart-button").get_attribute("data-id")
            link_scraper = producto_url = links_productos[pos][2]
            categoria = browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split("\'")[3]
            item.append([pos,producto_id,descripcion,categoria,color,precio,imagen,producto_link,link_scraper])
            clear_output()
            print(f"Escrapeando imagenes del prodcuto {pos + 1} / {len (links_productos)}")
    except:
            imagen = browser.find_element_by_class_name("fotorama__img").get_attribute("src")
            id_prodcuto = browser.find_element_by_class_name("base").text.split(" ")[1]
            descripcion = browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split('"')[21]
            #descripcion = browser.find_element_by_class_name("base").text.split(" ")[0]
#             try:
#                 color = browser.find_element_by_css_selector("span.swatch-attribute-selected-option").get_attribute("innerHTML")
#             except:
#                 color = "Sin color"
            color = browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text").split('"')[31]
            precio = browser.find_element_by_class_name("price").text
            producto_link = links_productos[pos][1]
            try:
                producto_id = browser.find_element_by_id("product-addtocart-button").get_attribute("data-id")
            except:
                producto_id = browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split('"')[1]
            link_scraper = links_productos[pos][2]
            categoria = browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split("\'")[3]
            item.append([pos,producto_id,descripcion,categoria,color,precio,imagen,producto_link,link_scraper])
            clear_output()
            print(f"Escrapeando imagenes del prodcuto {pos + 1} / {len (links_productos)}")
    pos = pos + 1
    if(pos == len (links_productos)):
        print("Fin del escrapeo")
        break

Escrapeando imagenes del prodcuto 139 / 543


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#product-options-wrapper > div > script"}
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-76-generic x86_64)


In [206]:
len(item)

11

In [207]:
item

[[0,
  '7943',
  'Teddy 7943',
  'Babydolls',
  'HUESO',
  '$399',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/4/0/4081126.jpg',
  'https://www.ilusion.com/babydolls/teddy-7943.html',
  'https://www.ilusion.com/babydolls.html'],
 [0,
  '7943',
  'Teddy 7943',
  'Babydolls',
  'HUESO',
  '$399',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/4/0/4081138.jpg',
  'https://www.ilusion.com/babydolls/teddy-7943.html',
  'https://www.ilusion.com/babydolls.html'],
 [1,
  '42188',
  'Babydoll 42188',
  'Babydolls',
  'RAMILLETE IV',
  '$299',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/4/0/4081142.jpg',
  'https://www.ilusion.com/babydolls/babydoll-42188.html',
  'https://www.ilusion.com/babydolls.html'],
 [2,
  '42187',
  'Babydoll 42187',
  'Babydolls',


In [135]:
links_productos[pos][1]

'https://www.ilusion.com/panties/panty-control-1489.html'

In [142]:
browser.current_url

'https://www.ilusion.com/lenceria-ropa-de-dormir/pijama-corta-satinada-3824.html'

In [143]:
browser.find_element_by_css_selector("div.swatch-attribute-options.clearfix").find_elements_by_tag_name("div")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.swatch-attribute-options.clearfix"}
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-76-generic x86_64)


In [162]:
item[350:]

[[114,
  '7317',
  'Bra',
  'PLUMAS II',
  '$171.60',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/3/8/3876370.jpg',
  'https://www.ilusion.com/bras/bra-push-up-con-encaje-7317.html',
  'https://www.ilusion.com/bras.html?p=8'],
 [115,
  '7308',
  'Bra',
  'AZUL INTENSO',
  '$171.60',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/3/6/3689429.jpg',
  'https://www.ilusion.com/bras/bra-push-up-con-encaje-7308.html',
  'https://www.ilusion.com/bras.html?p=8'],
 [115,
  '7308',
  'Bra',
  'CEREZA I',
  '$171.60',
  'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/3/8/3877084.jpg',
  'https://www.ilusion.com/bras/bra-push-up-con-encaje-7308.html',
  'https://www.ilusion.com/bras.html?p=8'],
 [115,
  '7308',
  'Bra',
  'JADE II',
  '$171.60',
  'https://diltex-c

In [36]:
browser.find_element_by_css_selector("#product-options-wrapper > div > script")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#product-options-wrapper > div > script"}
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-76-generic x86_64)


In [38]:
browser.get("https://www.ilusion.com/juvenil-trajes-de-ba-o/traje-de-bano-6999.html")

In [39]:
browser.current_url

'https://www.ilusion.com/juvenil-trajes-de-ba-o/traje-de-bano-6999.html'

In [43]:
browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text").split('"full"')

['\n    require(["jquery", "jquery/ui", "swatchRenderer"], function ($) {\n        $(\'.swatch-opt\').SwatchRenderer({\n            jsonConfig: {"attributes":{"90":{"id":"90","code":"color","label":"Color","options":[{"id":"11703","label":"ULTRAMARINO","products":["129955","129956","129957","129958"]},{"id":"12871","label":"ESTRELLA III","products":["129951","129952","129953","129954"]}]},"176":{"id":"176","code":"size","label":"Talla","options":[{"id":"12212","label":"CH","products":["129951","129956"]},{"id":"12241","label":"M","products":["129953","129957"]},{"id":"12217","label":"G","products":["129952","129955"]},{"id":"12284","label":"XG","products":["129954","129958"]}]}},"template":"$<%- data.price %>","optionPrices":{"129951":{"oldPrice":{"amount":"199"},"basePrice":{"amount":"169.15"},"finalPrice":{"amount":"169.15"}},"129952":{"oldPrice":{"amount":"199"},"basePrice":{"amount":"169.15"},"finalPrice":{"amount":"169.15"}},"129953":{"oldPrice":{"amount":"199"},"basePrice":{"amou

In [ ]:
for x in browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text"):
    

In [188]:
#descripcion 
browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split('"')[21]

'Babydoll 42188'

In [50]:
import re

texto = browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text")

urls = re.findall(https:\\\\.*?\.jpg+', texto)

In [51]:
urls

[]

In [49]:
https:\/\/diltex-cdn-endpoint.azureedge.net\/pub\/media\/catalog\/product\/cache\/1\/image\/700x700\/

[]

In [52]:
browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text")

'\n    require(["jquery", "jquery/ui", "swatchRenderer"], function ($) {\n        $(\'.swatch-opt\').SwatchRenderer({\n            jsonConfig: {"attributes":{"90":{"id":"90","code":"color","label":"Color","options":[{"id":"11703","label":"ULTRAMARINO","products":["129955","129956","129957","129958"]},{"id":"12871","label":"ESTRELLA III","products":["129951","129952","129953","129954"]}]},"176":{"id":"176","code":"size","label":"Talla","options":[{"id":"12212","label":"CH","products":["129951","129956"]},{"id":"12241","label":"M","products":["129953","129957"]},{"id":"12217","label":"G","products":["129952","129955"]},{"id":"12284","label":"XG","products":["129954","129958"]}]}},"template":"$<%- data.price %>","optionPrices":{"129951":{"oldPrice":{"amount":"199"},"basePrice":{"amount":"169.15"},"finalPrice":{"amount":"169.15"}},"129952":{"oldPrice":{"amount":"199"},"basePrice":{"amount":"169.15"},"finalPrice":{"amount":"169.15"}},"129953":{"oldPrice":{"amount":"199"},"basePrice":{"amoun

In [204]:
browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text").split('"thumb":')

['\n    require(["jquery", "jquery/ui", "swatchRenderer"], function ($) {\n        $(\'.swatch-opt\').SwatchRenderer({\n            jsonConfig: {"attributes":{"90":{"id":"90","code":"color","label":"Color","options":[{"id":"12882","label":"RAMILLETE IV","products":[]},{"id":"12886","label":"ROSAS VII","products":["124742","124743","124744","124745"]}]},"176":{"id":"176","code":"size","label":"Tallas","options":[{"id":"12212","label":"CH","products":["124744"]},{"id":"12241","label":"M","products":["124745"]},{"id":"12217","label":"G","products":["124743"]},{"id":"12284","label":"XG","products":["124742"]}]}},"template":"$<%- data.price %>","optionPrices":{"124742":{"oldPrice":{"amount":"299"},"basePrice":{"amount":"254.15"},"finalPrice":{"amount":"254.15"}},"124743":{"oldPrice":{"amount":"299"},"basePrice":{"amount":"254.15"},"finalPrice":{"amount":"254.15"}},"124744":{"oldPrice":{"amount":"299"},"basePrice":{"amount":"254.15"},"finalPrice":{"amount":"254.15"}},"124745":{"oldPrice":{"a

In [192]:
browser.find_element_by_css_selector("#product-options-wrapper > div > script").get_attribute("text").split('"')[31]

'RAMILLETE IV'

In [169]:
browser.current_url

'https://www.ilusion.com/moda.html'

In [166]:
browser.find_element_by_css_selector("span.swatch-attribute-selected-option")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.swatch-attribute-selected-option"}
  (Session info: headless chrome=79.0.3945.130)


In [159]:
browser.find_element_by_css_selector("div.swatch-attribute-options.clearfix").find_elements_by_tag_name("div")

[<selenium.webdriver.remote.webelement.WebElement (session="88c8bcfc8adf54ea6ece11405081e00e", element="df39918c-ef32-45d2-8192-798d7ddf1ea6")>]

In [30]:
browser.save_screenshot("asd.png")

True

In [144]:
producto_id = browser.find_element_by_id("product-addtocart-button").get_attribute("data-id")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"id","selector":"product-addtocart-button"}
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-76-generic x86_64)


In [150]:
browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split("\'")[3]

'Ropa de Dormir'

In [155]:
browser.find_element_by_css_selector("body > script:nth-child(6)").get_attribute("text").split('"')[1]

'3824'

In [163]:
browser.current_url

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=40061): Max retries exceeded with url: /session/4ebc050b20a76f1ad09fe9c64d1484e5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe92f21ce10>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [106]:
browser.find_element_by_class_name("fotorama__img").get_attribute("src")

'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/4/0/4081138.jpg'

In [80]:
soup = BeautifulSoup(requests.get(links_productos[0][1]).content, "html.parser")

In [88]:
soup.find_all("fotorama__stage")

[]

In [96]:
result = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['fotorama__stage__frame fotorama__active fotorama_vertical_ratio fotorama__loaded fotorama__loaded--img'])

In [97]:
result

[]

In [129]:
# descripcion
browser.find_element_by_class_name("base").text

'Babydoll 42184'

In [33]:
#Precio
browser.find_element_by_class_name("price").text

'$399'

In [133]:
#id
browser.find_element_by_class_name("base").text.split(" ")[1]

'42184'

In [143]:
#color nombre
browser.find_element_by_css_selector("span.swatch-attribute-selected-option").get_attribute("innerHTML")

'VERDE II'

In [205]:
2

2

In [95]:
browser.save_screenshot("asd.png")

True

In [94]:
browser.find_element_by_class_name("swatch-attribute.color").find_elements_by_class_name("swatch-option.image")[0].click()

In [102]:
browser.find_element_by_class_name("fotorama__img").get_attribute("src")

'https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/700x700/bcf7ce64d63d93f1f374273bba74938d/4/0/4079095.jpg'

In [ ]:
items = []
pos = 0
for 
for x in browser.find_element_by_class_name("swatch-attribute.color").find_elements_by_class_name("swatch-option.image"):
    x.click()
    image = browser.find_element_by_class_name("fotorama__img").get_attribute("src")
    items.append([pos,image])
    pos = pos + 1 

In [116]:
browser = createDriver()


url = 'https://www.ilusion.com/'


estilos = ['LENCERÍA','MODA','DEPORTIVA']


browser.get(url)
hrefs = []
elementos_a = browser.find_elements_by_tag_name('a')
for i in elementos_a:
    try:
        texto= i.find_element_by_tag_name('span').text
        if texto != "" and texto in estilos:
            hrefs.append([i.get_attribute('href'),
                          texto
                         ])
    except:
        pass

hrefs_categories = []
for item in hrefs:
    browser.get(item[0])
   
    categories = browser.find_element_by_class_name('toggle-nav-categories')
    for c in categories.find_elements_by_tag_name('a'):
       
        hrefs_categories.append([item[1],
                                c.get_attribute('href'),
                                c.text])

In [118]:
hrefs

[['https://www.ilusion.com/lenceria.html', 'LENCERÍA'],
 ['https://www.ilusion.com/moda.html', 'MODA'],
 ['https://www.ilusion.com/deportiva.html', 'DEPORTIVA']]

In [119]:
hrefs_categories

[['LENCERÍA', 'https://www.ilusion.com/babydolls.html', 'Babydolls'],
 ['LENCERÍA', 'https://www.ilusion.com/bras.html', 'Bras'],
 ['LENCERÍA',
  'https://www.ilusion.com/lenceria-tops-y-camisetas.html',
  'Tops y Camisetas'],
 ['LENCERÍA', 'https://www.ilusion.com/panties.html', 'Panties'],
 ['LENCERÍA',
  'https://www.ilusion.com/lenceria-accesorios.html',
  'Accesorios'],
 ['LENCERÍA', 'https://www.ilusion.com/fondos.html', 'Fondos'],
 ['LENCERÍA', 'https://www.ilusion.com/control.html', 'Control'],
 ['LENCERÍA',
  'https://www.ilusion.com/lenceria-ropa-de-dormir.html',
  'Ropa de Dormir'],
 ['LENCERÍA',
  'https://www.ilusion.com/lenceria-maternidad.html',
  'Maternidad'],
 ['MODA', 'https://www.ilusion.com/moda-lenceria.html', 'Lencería'],
 ['MODA',
  'https://www.ilusion.com/moda-ropa-de-dormir.html',
  'Ropa De Dormir'],
 ['MODA', 'https://www.ilusion.com/moda-ropa-exterior.html', 'Ropa Exterior'],
 ['MODA', 'https://www.ilusion.com/moda-calzado.html', 'Calzado'],
 ['MODA', 'htt

In [ ]:
for x in browser.find_elements_by_class_name("swatch-attribute-options.clearfix"):
    x.click()
    imagen = 

In [57]:
browser.find_elements_by_css_selector("gallery-placeholder")

[]

In [349]:
browser.get("https://www.ilusion.com/juvenil-ropa-exterior/top-6782.html")

In [374]:
files = []
data = str(soup.find_all("script")).split("\n")
for d in data:
    if '.jpg' in d:
        url = d.split(": ")[1].replace("'", "").replace(",", "")
        print(url)
        files.append(url)

{"attributes":{"90":{"id":"90""code":"color""label":"Color""options":[{"id":"11783""label":"VIOLETA""products":["102667""102669""102670"]}]}"176":{"id":"176""code":"size""label":"Talla""options":[{"id":"12212""label":"CH""products":["102669"]}{"id":"12241""label":"M""products":["102670"]}{"id":"12217""label":"G""products":[]}{"id":"12284""label":"XG""products":["102667"]}]}}"template":"$<%- data.price %>""optionPrices":{"102667":{"oldPrice":{"amount":"114.2"}"basePrice":{"amount":"114.2"}"finalPrice":{"amount":"114.2"}}"102669":{"oldPrice":{"amount":"114.2"}"basePrice":{"amount":"114.2"}"finalPrice":{"amount":"114.2"}}"102670":{"oldPrice":{"amount":"114.2"}"basePrice":{"amount":"114.2"}"finalPrice":{"amount":"114.2"}}}"prices":{"oldPrice":{"amount":"114.2"}"basePrice":{"amount":"114.2"}"finalPrice":{"amount":"114.2"}}"productId":"102671""chooseText":"Elegir una opci\u00f3n...""images":{"102667":[{"thumb":"https:\/\/diltex-cdn-endpoint.azureedge.net\/pub\/media\/catalog\/product\/cache\

In [355]:
url.split('full":"')[-1].split('""')[0]

'https:\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/bcf7ce64d63d93f1f374273bba74938d\\/3\\/8\\/3875655.jpg'

In [358]:
url.split

['[{"thumb"',
 '"https',
 '\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/thumbnail\\/90x90\\/aad7f8c9fa33d65d5d43285a0eed3116\\/3\\/8\\/3875655.jpg""img"',
 '"https',
 '\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/700x700\\/bcf7ce64d63d93f1f374273bba74938d\\/3\\/8\\/3875655.jpg""full"',
 '"https',
 '\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/bcf7ce64d63d93f1f374273bba74938d\\/3\\/8\\/3875655.jpg""caption"',
 '"""position"',
 '"0""isMain"',
 'true}]']

In [367]:
url.replace('[{"thumb"','').replace('""position"','').replace('"""position"','').replace('true}]','').replace('"img":"','').replace('"caption":":"0""isMain":','').replace('full','').replace(':','').split('"')

['',
 'https\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/thumbnail\\/90x90\\/aad7f8c9fa33d65d5d43285a0eed3116\\/3\\/8\\/3875655.jpg',
 'https\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/700x700\\/bcf7ce64d63d93f1f374273bba74938d\\/3\\/8\\/3875655.jpg',
 '',
 '',
 '',
 'https\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/bcf7ce64d63d93f1f374273bba74938d\\/3\\/8\\/3875655.jpg',
 '']

In [ ]:
https://diltex-cdn-endpoint.azureedge.net/pub/media/catalog/product/cache/1/image/bcf7ce64d63d93f1f374273bba74938d/4/0/4080440.jpg

In [254]:
soup = BeautifulSoup(requests.get("https://www.ilusion.com/juvenil-trajes-de-ba-o/traje-de-bano-6972.html").content ,'html.parser')

In [270]:
soup.find_all("script")[8].split("https:\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/")

TypeError: 'NoneType' object is not callable

In [305]:
soup

[<script>
    var require = {
        "baseUrl": "https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX"
    };
</script>, <script src="https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX/requirejs/require.js" type="text/javascript"></script>, <script src="https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX/mage/requirejs/mixins.js" type="text/javascript"></script>, <script src="https://www.ilusion.com/pub/static/_requirejs/frontend/Magento/ilusionmx/es_MX/secure/requirejs-config.js" type="text/javascript"></script>, <script src="https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX/js/js-ilusionmx.js" type="text/javascript"></script>, <script src="https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX/Diltex_Chat/js/hubs.js" type="text/javascript"></script>, <script src="https://www.ilusion.com/pub/static/frontend/Magento/ilusionmx/es_MX/Diltex_MarketingCloud/js/timer.js" type="text/javascript"></script>, <script

In [ ]:
https:\/\/diltex-cdn-endpoint.azureedge.net\/pub\/media\/catalog\/product\/cache\/1\/image\/

In [276]:
data = str(soup.find_all("script"))

In [291]:
import re

from bs4 import BeautifulSoup


pattern = re.compile(r'(http(s?):)([/|.|\w|\s|-])*\.(?:jpg|gif|png)')
soup = BeautifulSoup(data, "html.parser")

script = soup.find("script", text=pattern)

In [304]:
script

<script type="text/x-magento-init">
            {
            "[data-block='minicart']": {
            "Magento_Ui/js/core/app": {"components":{"minicart_content":{"children":{"subtotal.container":{"children":{"subtotal":{"children":{"subtotal.totals":{"config":{"display_cart_subtotal_incl_tax":0,"display_cart_subtotal_excl_tax":1,"template":"Magento_Tax\/checkout\/minicart\/subtotal\/totals"},"component":"Magento_Tax\/js\/view\/checkout\/minicart\/subtotal\/totals","children":{"subtotal.totals.msrp":{"component":"Magento_Msrp\/js\/view\/checkout\/minicart\/subtotal\/totals","config":{"displayArea":"minicart-subtotal-hidden","template":"Magento_Msrp\/checkout\/minicart\/subtotal\/totals"}}}}},"component":"uiComponent","config":{"template":"Magento_Checkout\/minicart\/subtotal"}}},"component":"uiComponent","config":{"displayArea":"subtotalContainer"}},"item.renderer":{"component":"uiComponent","config":{"displayArea":"defaultRenderer","template":"Magento_Checkout\/minicart\/item\/defaul

In [339]:
str(list(str(soup.find_all("script")).split("<script>"))[8].split('":"'))

'[\'\\n    require(["jquery", "jquery/ui", "swatchRenderer"], function ($) {\\n        $(\\\'.swatch-opt\\\').SwatchRenderer({\\n            jsonConfig: {"attributes":{"90":{"id\', \'90","code\', \'color","label\', \'Color","options":[{"id\', \'11783","label\', \'VIOLETA","products":["102667","102669","102670"]}]},"176":{"id\', \'176","code\', \'size","label\', \'Talla","options":[{"id\', \'12212","label\', \'CH","products":["102669"]},{"id\', \'12241","label\', \'M","products":["102670"]},{"id\', \'12217","label\', \'G","products":[]},{"id\', \'12284","label\', \'XG","products":["102667"]}]}},"template\', \'$<%- data.price %>","optionPrices":{"102667":{"oldPrice":{"amount\', \'114.2"},"basePrice":{"amount\', \'114.2"},"finalPrice":{"amount\', \'114.2"}},"102669":{"oldPrice":{"amount\', \'114.2"},"basePrice":{"amount\', \'114.2"},"finalPrice":{"amount\', \'114.2"}},"102670":{"oldPrice":{"amount\', \'114.2"},"basePrice":{"amount\', \'114.2"},"finalPrice":{"amount\', \'114.2"}}},"prices"

In [ ]:
https://www.ilusion.com/moda-ropa-de-dormir/camisaa-n-3761.html

In [239]:
for td in soup.find_all("div",text="https:\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/"):
    print(td.find_next_sibling("div"))

[]

In [241]:
script = soup.find("script",)

In [253]:
import re

from bs4 import BeautifulSoup

data = str(soup.find_all("script"))

soup = BeautifulSoup(data, "html.parser")

script = soup.find("script")
if script:
    match = pattern.search("https:\\/\\/diltex-cdn-endpoint.azureedge.net\\/pub\\/media\\/catalog\\/product\\/cache\\/1\\/image\\/")
    if match:
        email = match.group(1)
        print(email)

In [301]:
data = soup.find_all("script")

'https://www.ilusion.com/juvenil-trajes-de-ba-o.html'

In [22]:
2

2